In [14]:
import pandas as pd

In [15]:
df_tck = pd.read_csv('results/params/t_ckt.csv')
df_Tc = pd.read_csv('results/params/T_c.csv')
df_Tk = pd.read_csv('results/params/T_k.csv')
df_phi = pd.read_csv('../data/processed/city_rent.csv')
df_mj = pd.read_csv('results/params/major_params.csv')

df_tck = df_tck[df_tck['city_year'].astype(str).str.endswith('2019')]
df_Tc = df_Tc[df_Tc['city_year'].astype(str).str.endswith('2019')]
df_Tk = df_Tk[df_Tk['occ_year'].astype(str).str.endswith('2019')]
df_phi = df_phi[['city', '2019']]

df_phi = df_phi.rename(columns={'2019': 'phi'})

df_tck['city'] = df_tck['city_year'].str.split('_').str[0].str.replace(r'[()]', '', regex=True)
df_tck['occ'] = df_tck['occ_year'].str.extract(r'\((.*?)\)')[0]
df_Tc['city'] = df_Tc['city_year'].str.split('_').str[0].str.replace(r'[()]', '', regex=True)
df_Tk['occ'] = df_Tk['occ_year'].str.extract(r'\((.*?)\)')[0]

df_tck = df_tck[['city', 'occ', 't_ckt']]
df_Tc = df_Tc[['city', 'T_c']]
df_Tk = df_Tk[['occ', 'T_k']]

In [16]:
lambda_k = df_tck.groupby('occ').sum().reset_index()
lambda_k = lambda_k.drop(columns=['city'])
lambda_k = lambda_k[~lambda_k['occ'].str.contains('city_total')]
lambda_k = lambda_k.rename(columns={'t_ckt': 'lambda_k'})

In [17]:
z_c = df_Tc.copy()
z_c = z_c.merge(df_phi, on='city')
z_c['z_c'] = z_c['T_c'] / z_c['phi']
z_c = z_c.drop(columns=['T_c', 'phi'])
z_c['z_c'] = z_c['z_c'].apply(lambda x: x ** df_mj['theta'].values[0])

In [18]:
pi_ck = df_tck.copy()
pi_ck = pi_ck.merge(z_c, on='city')
pi_ck['t_ckt'] = pi_ck['t_ckt'] * pi_ck['z_c']
pi_ck = pi_ck.drop(columns=['z_c'])

pi_ck['t_ckt'] = pi_ck['t_ckt'].apply(lambda x: x ** (1 / (1 - df_mj['rho'][0])))
pi_ck = pi_ck.merge(lambda_k, on='occ')
pi_ck['pi_ck'] = pi_ck['t_ckt'] / pi_ck['lambda_k']
pi_ck = pi_ck.drop(columns=['t_ckt', 'lambda_k'])
pi_ck = pi_ck.pivot(index='city', columns='occ', values='pi_ck').reset_index()

In [19]:
pi_c = df_tck.copy()
pi_c = pi_c.merge(z_c, on='city')
pi_c['t_ckt'] = pi_c['t_ckt'] * pi_c['z_c']
pi_c = pi_c.drop(columns=['z_c'])
pi_c = pi_c.merge(lambda_k, on='occ')

omega = df_Tk.copy()
omega = omega.merge(lambda_k, on='occ')
omega['lambda_k'] = omega['lambda_k'].apply(lambda x: x ** (1 - df_mj['rho'][0]))
omega['T_k'] = omega['T_k'] * omega['lambda_k']
omega = omega.drop(columns=['lambda_k'])
omega['omega'] = omega['T_k'] / omega['T_k'].sum()
omega = omega.drop(columns=['T_k'])

pi_c['t_ckt'] = pi_c['t_ckt'].apply(lambda x: x ** (1 / (1 - df_mj['rho'][0])))
pi_c['t_ckt'] = pi_c['t_ckt'] / pi_c['lambda_k']
pi_c = pi_c.drop(columns=['lambda_k'])
pi_c = pi_c.merge(omega, on='occ')
pi_c['pi_c'] = pi_c['t_ckt'] * pi_c['omega']
pi_c = pi_c.drop(columns=['t_ckt', 'omega'])
pi_c = pi_c.groupby('city').sum().reset_index()
pi_c = pi_c.drop(columns=['occ'])

In [20]:
df_tck = df_tck.pivot(index='city', columns='occ', values='t_ckt').reset_index()